# HOMEWORK 3

In [1]:
def install(package):
    try:
        import subprocess
        import sys
        subprocess.call([sys.executable, "-m", "pip", "install", package])
        print("Library installed")
    except:
        print("Cannot install the library")

In [2]:
try:
    from langdetect import detect
except:
    print("The library LanfDetect is not in the repository, I will install it")
    install('langdetect')
    from langdetect import detect

try:
    from texttable import Texttable
except:
    print("The library TextTable is not in the repository, I will install it")
    install('texttable')
    from texttable import Texttable


In [3]:
import pandas as pd
import unicodecsv as csv # We also import the unicodecsv library to handle any Unicode objects in the CSV report...as csv.
import nltk
import string

from nltk import word_tokenize
from nltk.corpus import stopwords
from langdetect import detect

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabiomontello/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

STEP 1 - DATA

In [4]:
DIR = 'data/docs/' # C:/Users/david/Desktop/DATA SCIENCE/AMD/documents/

df = pd.read_csv("data/Airbnb_Texas_Rentals.csv", sep=",") # we read the dataset

In [5]:
df = df.drop('Unnamed: 0',axis = 1) #this column is useless for our analisys
df = df[pd.notnull((df['description'])) & (df['description'] != "\\" ) & (df['description'] != "." )] #If the row has no description, for us is useless, so we drop it
df = df.replace(r'\\n','', regex=True) #Remove all the new lines in the dataframe
df['language'] = 'nd'
df.head()

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,language
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...,nd
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...,nd
2,$59,1,Houston,January 2017,'River house on island close to the city' A we...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...,nd
3,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...,nd
4,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...,nd


STEP 2 - WRITE DOCUMENTS

In [ ]:
for row in df.iterrows():
    file_name = DIR + 'doc_'+str(row[0])+'.tsv'
    with open(file_name,'wb') as file: # the wb means we are writing a file in binary mode.
        csv_writer = csv.writer(file, delimiter = '\t') #now we are writing the files as tab separated files.
        csv_writer.writerow(list(row[1].values)) #writerow want a list object ad input, so we used list.
        file.close() #Once the file is written, we close it, so we free some memory


In [7]:
docdict = {}

languages = {'en': 'english', 'de': 'german', 'es': 'spanish', 'nl': 'dutch'}

for i, row in df[:].iterrows(): #We limit to 5 just to work quickly

    #Remove punctuation
    text = str(row['description']).translate({key: None for key in string.punctuation}) + ' '+ str(row['title']).translate({key: None for key in string.punctuation})
    
    if(text):
        try:
            lan = detect(str(text))
            if (lan in languages):
                languages[lan]
        
                #The text here is Spanish, we can decide what to do
                df.loc[i,'language'] = lan
        
                #Remove all the stopwords and all the non alphabetic words
                tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
        
                #Stem all the words
                docdict[i] = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
            else:
                pass
        except:
            pass

# languages
docdict

{0: ['welcom',
  'stay',
  'privat',
  'room',
  'queen',
  'bed',
  'detach',
  'privat',
  'bathroom',
  'second',
  'anoth',
  'privat',
  'bedroom',
  'sofa',
  'bed',
  'avail',
  'addit',
  'addit',
  'iah',
  'airportairport',
  'avail',
  'privat',
  'iah',
  'airport'],
 1: ['fulli',
  'remodel',
  'home',
  'upscal',
  'nw',
  'alamo',
  'height',
  'amaz',
  'locat',
  'hous',
  'conveni',
  'locat',
  'quiet',
  'beauti',
  'season',
  'prestigi',
  'neighborhood',
  'close',
  'loop',
  'featur',
  'open',
  'floor',
  'origin',
  'hardwood',
  'full',
  'bathroom',
  'independ',
  'room',
  'sleep',
  'moreeuropean',
  'inspir',
  'kitchen',
  'driveway',
  'park',
  'uniqu',
  'alamo',
  'height',
  'design',
  'inspir'],
 2: ['hous',
  'island',
  'close',
  'well',
  'maintain',
  'river',
  'hous',
  'san',
  'jacinto',
  'river',
  'extra',
  'room',
  'temporari',
  'river',
  'hous',
  'near',
  'citi'],
 3: ['privat',
  'bedroom',
  'cute',
  'littl',
  'home',
  

In [8]:
voc = {}
cont = 0
for key, value in docdict.items():
    for elem in value:
        if(elem not in voc):
            voc[elem] = cont
            cont += 1
print(voc)

{'welcom': 0, 'stay': 1, 'privat': 2, 'room': 3, 'queen': 4, 'bed': 5, 'detach': 6, 'bathroom': 7, 'second': 8, 'anoth': 9, 'bedroom': 10, 'sofa': 11, 'avail': 12, 'addit': 13, 'iah': 14, 'airportairport': 15, 'airport': 16, 'fulli': 17, 'remodel': 18, 'home': 19, 'upscal': 20, 'nw': 21, 'alamo': 22, 'height': 23, 'amaz': 24, 'locat': 25, 'hous': 26, 'conveni': 27, 'quiet': 28, 'beauti': 29, 'season': 30, 'prestigi': 31, 'neighborhood': 32, 'close': 33, 'loop': 34, 'featur': 35, 'open': 36, 'floor': 37, 'origin': 38, 'hardwood': 39, 'full': 40, 'independ': 41, 'sleep': 42, 'moreeuropean': 43, 'inspir': 44, 'kitchen': 45, 'driveway': 46, 'park': 47, 'uniqu': 48, 'design': 49, 'island': 50, 'well': 51, 'maintain': 52, 'river': 53, 'san': 54, 'jacinto': 55, 'extra': 56, 'temporari': 57, 'near': 58, 'citi': 59, 'cute': 60, 'littl': 61, 'situat': 62, 'covet': 63, 'garden': 64, 'acr': 65, 'access': 66, 'campus': 67, 'recent': 68, 'purchas': 69, 'mile': 70, 'outsid': 71, 'downtown': 72, 'fort

In [10]:
invidx = {} # 1: [0,2,5], 245: [1, 4, 7]

for key, value in docdict.items():
    #
    for elem in value:
        if(voc[elem] in invidx):
            invidx[voc[elem]] = invidx[voc[elem]] + [key]
        else:
            invidx[voc[elem]] = [key]
print(invidx)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
# def jaccard_sim(list1, list2):
#     a = list1
#     b = list2
#     c = a.intersection(b)
#     return float(len(c))/ len(a) + len(b) - len(c)
def searchJac(string, n):
    languages = {'en': 'english', 'de': 'german', 'es': 'spanish', 'nl': 'dutch'}
    text = string
    lan = detect(str(text))
    if(lan not in languages):
        lan = 'en'
    tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
    result = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
    
    listofdocs = {}
    for elem in result:
        if(elem in voc):
            if(len(listofdocs) == 0):
                listofdocs = set(invidx[voc[elem]])
            else:
                listofdocs = listofdocs & set(invidx[voc[elem]])
    t = Texttable()
    t.add_row(['Title', 'Description', 'City', 'URL'])
    for elem in listofdocs:
        file_name = DIR + 'doc_'+str(elem)+'.tsv'
        with open(file_name,'rb') as file: # the wb means we are writing a file in binary mode.
            text = str(file.read()).split('\\t')
            t.add_row([text[7], text[4], text[2], text[8]])

    print(t.draw())

In [16]:
searchJac('Increíble habitación lugares', 10)

+---------------------+---------------------+-------------+--------------------+
| Title               | Description         | City        | URL                |
+---------------------+---------------------+-------------+--------------------+
| Cuarto luminoso     | Lugares de          | Irving      | https://www.airbnb |
|                     | inter\xc3\xa9s:     |             | .com/rooms/1607339 |
|                     | transporte          |             | 3?location=Coppell |
|                     | p\xc3\xbablico,     |             | %2C%20TX\r\n'      |
|                     | parques. Te va a    |             |                    |
|                     | encantar mi espacio |             |                    |
|                     | porque Hay mucho    |             |                    |
|                     | verde y espacios    |             |                    |
|                     | con juegos para     |             |                    |
|                     | ni\x

In [13]:
df.loc[df['language'] == 'es']

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,language
113,$45,1,San Antonio,August 2015,Lugares de interés: el aeropuerto se encuentra...,29.639149,-98.429010,Peaceful home 25 min from Airport &35 min Down...,https://www.airbnb.com/rooms/15773671?location...,es
121,$400,2,Houston,January 2017,Lugares de interés: Hobby airport. Te va a enc...,29.692171,-95.278347,Houston 15 min NRG,https://www.airbnb.com/rooms/16973563?location...,es
2480,$800,3,Coppell,March 2016,La mejor zona de Dallas a 15 min del aeropuert...,32.987147,-96.968529,"Lujo,Comodidad,Relax y Shopping!!!",https://www.airbnb.com/rooms/13570726?location...,es
2618,$70,1,Manor,April 2015,Es una casa de dos pisos ubicada en Manor Tx. ...,30.352237,-97.522520,Cuarto en renta,https://www.airbnb.com/rooms/5851805?location=...,es
2731,$25,1,San Antonio,November 2016,pequeno condominio con una recamara grande y b...,29.529369,-98.435646,"Private Room and Bath, Welcome",https://www.airbnb.com/rooms/19434216?location...,es
3307,$40,1,Katy,February 2016,Habitación con todas las comodidades. Todo lo ...,29.771096,-95.760388,Habitación ejecutiva numero dos (2),https://www.airbnb.com/rooms/12386464?location...,es
3837,$130,2,San Antonio,August 2016,"Lugares de interés: Six Flags, Sea World, San ...",29.516285,-98.423481,"Excellent option in San Antonio, great location",https://www.airbnb.com/rooms/14529034?location...,es
3902,$125,4,Conroe,April 2017,"Increíble casa cerca de The Woodlands, acogedo...",30.217483,-95.439574,Incredible Cozy Home in the Woodlands,https://www.airbnb.com/rooms/18811007?location...,es
5252,$100,1,Conroe,May 2013,"A 5 mins del Mall de Woodlands, a 20 mins del ...",30.216015,-95.429937,Perfecto para venir de compras,https://www.airbnb.com/rooms/1617410?location=...,es
5675,$25,1,Irving,March 2014,"Lugares de interés: transporte público, parque...",32.861782,-96.988759,Cuarto luminoso,https://www.airbnb.com/rooms/16073393?location...,es
